In [2]:
import os
import json
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models, callbacks

In [4]:
DATA_DIR = "/kaggle/input/cattle-breed-recognition/dataset"
if not os.path.exists(DATA_DIR):
    raise FileNotFoundError(f"Dataset not found at {DATA_DIR}. Attach the Kaggle dataset before running.")

In [16]:
IMG_SIZE = (224, 224)
BATCH_SIZE = 32
SEED = 42
BASE_EPOCHS = 10
FINE_TUNE_EPOCHS = 10
SAVED_MODEL_DIR = "saved_model/breed_classifier"
os.makedirs(SAVED_MODEL_DIR, exist_ok=True)

In [7]:
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    os.path.join(DATA_DIR, "train"),
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    seed=SEED,
    label_mode="int",
    shuffle=True
)
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    os.path.join(DATA_DIR, "val"),
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    seed=SEED,
    label_mode="int",
    shuffle=False
)
test_ds = tf.keras.preprocessing.image_dataset_from_directory(
    os.path.join(DATA_DIR, "test"),
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    seed=SEED,
    label_mode="int",
    shuffle=False
)

class_names = train_ds.class_names
num_classes = len(class_names)
with open(os.path.join(SAVED_MODEL_DIR, "classes.json"), "w") as f:
    json.dump(class_names, f)

Found 4129 files belonging to 41 classes.
Found 869 files belonging to 41 classes.
Found 928 files belonging to 41 classes.


In [8]:
AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.cache().shuffle(1000).prefetch(AUTOTUNE)
val_ds = val_ds.cache().prefetch(AUTOTUNE)
test_ds = test_ds.cache().prefetch(AUTOTUNE)

data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.12),
    layers.RandomZoom(0.12),
    layers.RandomContrast(0.12),
    layers.RandomTranslation(0.08, 0.08),
], name="data_augmentation")


In [9]:
base_model = tf.keras.applications.MobileNetV2(
    input_shape=IMG_SIZE + (3,), include_top=False, weights="imagenet"
)
base_model.trainable = False

inputs = tf.keras.Input(shape=IMG_SIZE + (3,))
x = data_augmentation(inputs)
x = tf.keras.applications.mobilenet_v2.preprocess_input(x)
x = base_model(x, training=False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(0.3)(x)
outputs = layers.Dense(num_classes, activation="softmax")(x)
model = tf.keras.Model(inputs, outputs)

9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [19]:
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ data_augmentation (Sequential)  │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ true_divide (TrueDivide)        │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ subtract (Subtract)             │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ mobilenetv2_1.00_224            │ (None, 7, 7, 1280)     │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 1280)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 41)             │        52,521 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 5,778,237 (22.04 MB)

 Trainable params: 1,733,865 (6.61 MB)

 Non-trainable params: 576,640 (2.20 MB)

 Optimizer params: 3,467,732 (13.23 MB)

In [10]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)


In [11]:
checkpoint_path = os.path.join(SAVED_MODEL_DIR, "best_model.h5")
cp = callbacks.ModelCheckpoint(checkpoint_path, monitor="val_accuracy", save_best_only=True, verbose=1)
es = callbacks.EarlyStopping(monitor="val_accuracy", patience=4, restore_best_weights=True, verbose=1)
reduce_lr = callbacks.ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=2, min_lr=1e-6, verbose=1)


In [12]:
model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=BASE_EPOCHS,
    callbacks=[cp, es, reduce_lr]
)

Epoch 1/10


I0000 00:00:1758006753.703279     105 cuda_dnn.cc:529] Loaded cuDNN version 90300


129/130 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step - accuracy: 0.1181 - loss: 3.6903
Epoch 1: val_accuracy improved from -inf to 0.31991, saving model to saved_model/breed_classifier/best_model.h5
130/130 ━━━━━━━━━━━━━━━━━━━━ 57s 120ms/step - accuracy: 0.1190 - loss: 3.6833 - val_accuracy: 0.3199 - val_loss: 2.4920 - learning_rate: 0.0010
Epoch 2/10
130/130 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step - accuracy: 0.2983 - loss: 2.5552
Epoch 2: val_accuracy improved from 0.31991 to 0.36364, saving model to saved_model/breed_classifier/best_model.h5
130/130 ━━━━━━━━━━━━━━━━━━━━ 8s 63ms/step - accuracy: 0.2984 - loss: 2.5547 - val_accuracy: 0.3636 - val_loss: 2.2801 - learning_rate: 0.0010
Epoch 3/10
130/130 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step - accuracy: 0.3363 - loss: 2.2815
Epoch 3: val_accuracy improved from 0.36364 to 0.39816, saving model to saved_model/breed_classifier/best_model.h5
130/130 ━━━━━━━━━━━━━━━━━━━━ 8s 63ms/step - accuracy: 0.3364 - loss: 2.2812 - val_accuracy: 0.3982 - val_loss: 2.1313 - le

In [13]:
base_model.trainable = True
fine_tune_at = int(len(base_model.layers) * 0.75)
for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False
for layer in base_model.layers[fine_tune_at:]:
    layer.trainable = True

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

In [14]:
ft_checkpoint = os.path.join(SAVED_MODEL_DIR, "best_model_finetuned.h5")
ft_cp = callbacks.ModelCheckpoint(ft_checkpoint, monitor="val_accuracy", save_best_only=True, verbose=1)
ft_es = callbacks.EarlyStopping(monitor="val_accuracy", patience=5, restore_best_weights=True, verbose=1)
ft_reduce = callbacks.ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=2, min_lr=1e-7, verbose=1)

In [17]:
model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=FINE_TUNE_EPOCHS,
    callbacks=[ft_cp, ft_es, ft_reduce]
)


Epoch 1/10
130/130 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step - accuracy: 0.4760 - loss: 1.7818
Epoch 1: val_accuracy did not improve from 0.45915
130/130 ━━━━━━━━━━━━━━━━━━━━ 12s 92ms/step - accuracy: 0.4760 - loss: 1.7817 - val_accuracy: 0.4534 - val_loss: 1.9239 - learning_rate: 1.0000e-05
Epoch 2/10
130/130 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step - accuracy: 0.4957 - loss: 1.7262
Epoch 2: val_accuracy did not improve from 0.45915
130/130 ━━━━━━━━━━━━━━━━━━━━ 12s 93ms/step - accuracy: 0.4957 - loss: 1.7261 - val_accuracy: 0.4545 - val_loss: 1.9170 - learning_rate: 1.0000e-05
Epoch 3/10
130/130 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step - accuracy: 0.4946 - loss: 1.7048
Epoch 3: val_accuracy did not improve from 0.45915
130/130 ━━━━━━━━━━━━━━━━━━━━ 12s 95ms/step - accuracy: 0.4946 - loss: 1.7045 - val_accuracy: 0.4534 - val_loss: 1.8947 - learning_rate: 1.0000e-05
Epoch 4/10
130/130 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step - accuracy: 0.5142 - loss: 1.6245
Epoch 4: val_accuracy did not improve from 0.45915
130/130

In [20]:
model.save("saved_model/breed_classifier/best_model_finetuned.h5")